# Section Schedule

Schedule students to sections based on their preferences.

## Setup

Import the required packages into the namespace.

In [ ]:
import difflib
import os

import numpy as np
import pandas as pd

import itertools
from typing import NamedTuple

In [ ]:
CAPACITIES = {
    'cs61a': 4,
    'cs61b': 6,
    'cs70' : 6,
    'ee16a': 6,
}
COURSE = 'cs61a'

In [ ]:
def path(filename, directory=COURSE):
    return os.path.join(directory, filename)

In [ ]:
SEED = sum(ord(c) for c in 'Computer Science Mentors')

## Section schedule

In [ ]:
CODE   = 'Code'
EMAIL  = 'Email Address'
COURSE = 'Course'
ROOM   = 'Room'
CAP    = 'Capacity'
TIME   = 'Time'

Import an existing schedule, if it exists.

Section data should be specified in the format,

```
'Email Address', 'Course', 'Room', 'Capacity', 'Time'
```

Capacity is important as we need to determine how many students can enroll into that section. If no capacity for a room is provided (or a non-integer capacity), then we will use the default capacity specified above per course.

### Using an existing schedule

In [ ]:
section_schedule = pd.read_csv(path('section-schedule.csv'), dtype=str).set_index(CODE)
section_schedule.head()

### Generating control codes

In [ ]:
def generate_control_code(row, length=6):
    """Return a control code of the desired length, zero-padded as necessary."""
    return str(abs(hash(tuple(row))))[:length].zfill(length)

In [ ]:
section_schedule = pd.read_csv(path('room-schedule.csv'))
section_schedule[CODE] = section_schedule.apply(generate_control_code, axis=1, raw=True)
section_schedule = section_schedule.set_index(CODE)
section_schedule.head()

#### Manually define a few sections

In [ ]:
manual_schedule = pd.DataFrame.from_records(
    [
        # ('Email', 'Course', 'Room', Capacity, 'Time'),
    ],
    columns=[EMAIL, COURSE, ROOM, CAP, TIME]
)

manual_schedule[CODE] = manual_schedule.apply(generate_control_code, axis=1, raw=True)
manual_schedule = manual_schedule.set_index(CODE)
manual_schedule

#### Export schedule

In [ ]:
section_schedule[CODE].to_csv(path('control-codes.csv'), index=False)

In [ ]:
section_schedule.to_csv(path('section-schedule.csv'))

## Input data

Load student preferences from a Google Form.

**The data must be downloaded directly from the Form, rather than a linked Google Sheet so that data is properly quoted.**

In [ ]:
EMAIL  = 'Username'
COURSE = 'Course'
FIRST  = 'First option'
SECOND = 'Second option'
THIRD  = 'Third option'
BACKUP = 'Backup options'
RANKS  = [FIRST, SECOND, THIRD]

SPLIT_ON = r', ?'

In [ ]:
preferences = pd.read_csv(path('preferences.csv'), dtype=str)
preferences = pd.concat([
    preferences[[EMAIL, COURSE] + RANKS],
    preferences[BACKUP].str.split(SPLIT_ON, expand=True).fillna('').astype(str)
], axis=1).rename(columns=str).set_index(EMAIL)
preferences.head()

### Enrollment priority

Give enrollment priority to a subset of the students.

In [ ]:
EMAIL = 'Email Address'
PREF  = 'Preferred'

In [ ]:
priority = pd.read_csv(path('priority.csv'), dtype=str)['Email']
preferences.insert(1, PREF, preferences.index.isin(priority))
preferences[preferences[PREF] == True].head()

## Greedy algorithm

Solve the problem using a simple greedy algorithm with randomized restarts.

In [ ]:
class Solution(NamedTuple):
    """Solution to an assignment problem."""
    assignments: dict
    stats: dict

    def metric(self, weights={FIRST: 3, SECOND: 2, THIRD: 1}):
        """Assign weights to each rank to evaluate the quality of the solution."""
        return sum(count * weights[rank] for rank, count in self.stats.items())

class Assignment(NamedTuple):
    email: str
    course: str

def generate_preference_slice(preferences, first=FIRST, index=1):
    return slice(pd.Index(preferences.columns).get_loc(first.lower()) + index,
                 len(preferences.columns))

### Validate the solution

In [ ]:
def validate(preferences, schedule, ranks=RANKS, preference_slice=None):
    """Validate the preferences to check for errors in student input."""
    preferences = preferences.rename(columns=str.lower)
    schedule = schedule.rename(columns=str.lower)
    if preference_slice is None:
        preference_slice = generate_preference_slice(preferences)
    def closest(key):
        match = difflib.get_close_matches(key, schedule.index, n=1)
        return match[0] if match else key
    invalid = []
    for row in preferences.itertuples():
        for rank, preference in itertools.zip_longest(ranks, row[preference_slice]):
            if not preference:
                continue
            elif preference not in schedule.index:
                print(f'{row.Index}: {preference} not found in schedule')
                invalid += [(row.Index, preference, closest(preference))]
            elif row.course != schedule.loc[preference].course:
                print(f'{row.Index}: {course} not found')
                invalid += [(row.Index, preference, closest(preference))]
    return pd.DataFrame.from_records(invalid, columns=['Email', 'Input', 'Match'])

# TODO: Write a function to replace invalid entries in the preferences with their match.

In [ ]:
validate(preferences, section_schedule)

In [ ]:
LIMIT = 1000
rand = np.random.RandomState(SEED)

In [ ]:
def greedy(preferences, schedule, ranks=RANKS,default_cap=CAPACITIES[COURSE],
           preference_slice=None):
    """Return a naive greedy algorithm for assigning each student in the preferences list
    to a section in the schedule based on the ranks.
    """
    preferences = preferences.rename(columns=str.lower)
    schedule = schedule.rename(columns=str.lower)
    if preference_slice is None:
        preference_slice = generate_preference_slice(preferences)
    if CAP in schedule.index:
        enrolled = {code: capacity for code, capacity in schedule[[CAP]].itertuples()}
    else:
        enrolled = {code: 4 for code in schedule.index}
    assignments = {}
    stats = {rank: 0 for rank in ranks}
    for row in preferences.itertuples():
        assignment = Assignment(row.Index, row.course)
        if assignment not in assignments:
            for rank, preference in itertools.zip_longest(ranks, row[preference_slice]):
                if (preference in schedule.index
                    and row.course == schedule.loc[preference].course
                    and enrolled[preference] > 0):
                    # Make an assignment if the preference exists, matches the course, and
                    # if there is space still left in the section
                    assignments[assignment] = preference
                    if rank in stats:
                        stats[rank] += 1
                    enrolled[preference] -= 1
                    break
    return Solution(assignments, stats)

def sample(preferences, priority=None):
    """Resample the preferences, prioritizing by True/False column value."""
    if priority is None:
        return preferences.sample(frac=1, random_state=rand)
    return (preferences[preferences[priority]]
            .sample(frac=1, random_state=rand)
            .append(preferences[~preferences[priority]]
                    .sample(frac=1, random_state=rand)))

In [ ]:
best = max((greedy(sample(preferences, priority=PREF), section_schedule)
            for _ in range(LIMIT)), key=Solution.metric)
best.stats

In [ ]:
len(best.assignments)

## Simulated Annealing

Implement a simulated annealing algorithm to improve upon the best greedy solution.

In [ ]:
# TODO: Implement simulated annealing algorithm

### Export schedule

In [ ]:
schedule = pd.DataFrame.from_records((
    (assignment.email, section) + tuple(section_schedule.loc[section])
    for assignment, section in best.assignments.items()
), columns=['Student Email', 'Section', 'Mentor Email','Course', 'Room', 'Capacity', 'Time'])

In [ ]:
schedule.to_csv(path('schedule.csv'), index=False)